In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv('/content/drive/My Drive/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')
    max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [4]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
 - 26s - loss: 0.8293 - accuracy: 0.6385
Epoch 2/2
 - 25s - loss: 0.6869 - accuracy: 0.7076
score 0.7486243940769403
accuracy 0.6806465983390808
Model Metrics Names ['loss', 'accuracy']


In [5]:
model.save('Sreevalli.h5')
from keras.models import load_model
model= load_model('Sreevalli.h5')
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0
Negative


In [6]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 38s - loss: 0.8312 - accuracy: 0.6454
Epoch 1/1
 - 37s - loss: 0.8246 - accuracy: 0.6450
Epoch 1/1
 - 38s - loss: 0.8298 - accuracy: 0.6376
Epoch 1/1
 - 39s - loss: 0.8311 - accuracy: 0.6390
Epoch 1/1
 - 37s - loss: 0.8272 - accuracy: 0.6426
Epoch 1/2
 - 37s - loss: 0.8313 - accuracy: 0.6437
Epoch 2/2
 - 37s - loss: 0.6929 - accuracy: 0.7121
Epoch 1/2
 - 36s - loss: 0.8283 - accuracy: 0.6426
Epoch 2/2
 - 37s - loss: 0.6916 - accuracy: 0.7063
Epoch 1/2
 - 38s - loss: 0.8330 - accuracy: 0.6403
Epoch 2/2
 - 38s - loss: 0.6818 - accuracy: 0.7116
Epoch 1/2
 - 37s - loss: 0.8320 - accuracy: 0.6421
Epoch 2/2
 - 37s - loss: 0.6833 - accuracy: 0.7106
Epoch 1/2
 - 37s - loss: 0.8263 - accuracy: 0.6487
Epoch 2/2
 - 37s - loss: 0.6762 - accuracy: 0.7119
Epoch 1/1
 - 26s - loss: 0.8409 - accuracy: 0.6443
Epoch 1/1
 - 26s - loss: 0.8304 - accuracy: 0.6427
Epoch 1/1
 - 26s - loss: 0.8356 - accuracy: 0.6406
Epoch 1/1
 - 26s - loss: 0.8362 - accuracy: 0.6413
Epoch 1/1
 - 26s - loss: 0.8274

In [9]:
data1 = pd.read_csv('/content/drive/My Drive/spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 52s - loss: 0.1694 - accuracy: 0.9400
Epoch 2/5
 - 51s - loss: 0.0416 - accuracy: 0.9885
Epoch 3/5
 - 50s - loss: 0.0230 - accuracy: 0.9941
Epoch 4/5
 - 51s - loss: 0.0111 - accuracy: 0.9971
Epoch 5/5
 - 50s - loss: 0.0116 - accuracy: 0.9962
0.0875708267716625
0.9825992584228516
['loss', 'accuracy']
